In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size =8
batch_size =4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cpu


In [34]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text=f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [35]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [36]:
n =int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix= torch.randint(len(data)- block_size, (batch_size,))
    x= torch.stack([data[i:i+block_size] for i in ix])
    y= torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x,y

x,y = get_batch('train')
print("inputs:")
print(x)
print("targets")
print(y)
      
                    


inputs:
tensor([[ 1, 54, 67,  1, 54, 76, 59, 74],
        [60, 60, 78,  9,  1, 73, 76, 68],
        [56, 68, 69, 58,  9,  1, 76, 61],
        [ 1, 55, 58,  1, 54, 55, 65, 58]])
targets
tensor([[54, 67,  1, 54, 76, 59, 74, 65],
        [60, 78,  9,  1, 73, 76, 68,  1],
        [68, 69, 58,  9,  1, 76, 61, 62],
        [55, 58,  1, 54, 55, 65, 58,  1]])


In [37]:
block_size=8

x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target=y[t]
    print(' whne input is',context,'target is', target)

 whne input is tensor([80]) target is tensor(1)
 whne input is tensor([80,  1]) target is tensor(1)
 whne input is tensor([80,  1,  1]) target is tensor(28)
 whne input is tensor([80,  1,  1, 28]) target is tensor(39)
 whne input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
 whne input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
 whne input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
 whne input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [38]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [41]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets= None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss=None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits= logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


4qzq6DQTV"[ 3sbEJtORpcY2 ;YAa﻿wv?K*Jj90sDNeI
bT8QINHD*.2-5x8rbXTGgCR43Zpi4qZCxHCw8g[zv]YHDQ'NfgW0XNv.'DIVwGWbpBlXy&gP.'ynuFm6Fa﻿o3f.h50CR8KaagQ'r4L98DFFnAbrldTrb7]﻿S5]]yusC?E-aNBTBx.)QVNnfJDBQ'CR8ZtE2KUngId1a8KpW0dI77-[4s(?dTLM )usIE2p.Z]:xgF?7QQUDBaB:)5qR,yV56.(mglH[n!oFImza6km3j&V"
mRTs!09b
GYJ?kWv.'&XzvJ]fS.l
csk?eeU;e,Jj)i﻿v:vIL?nDC;Q9w8i*!fs"YDgOqfIimz4vIbZ]vf.)1Xz?,g'r* 1eIp;RsKvGvi*ebcxv.Cz
YSbXz0N;YsM;dxg8(SM5TzlqRcYv&Daa-B8DFvLrp46kaVcEWrdTJUzPP_bGzEC(DFN2QQfK)CfLmvF":KVqL1PV﻿
k!7QLrEd.


In [45]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step: {iter}, train loss {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    xb,yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none= True)
    loss.backward()
    optimizer.step()
print(loss.item())

Step: 0, train loss 4.7539, val loss: 4.7562
Step: 250, train loss 4.6616, val loss: 4.7167
Step: 500, train loss 4.6279, val loss: 4.6412
Step: 750, train loss 4.5332, val loss: 4.5843
4.624715328216553
